In [1]:
import networkx as nx
import seaborn as sns 
from IPython.display import IFrame, display # TODO understand how to use visualization without it
import numpy as np
from datetime import datetime
import pandas as pd
import json
from functools import wraps

In [2]:
__TEMPLATE__ = """

<!DOCTYPE html>
<meta charset="cp1251">
<style>

circle {{
    fill: rgba(200, 200, 200, 0.5);
    stroke: rgba(200, 200, 200, 0.5);
    stroke-width: 5.0px;
}}

.circle.source_node {{
    fill: #f3f310;
}}

.circle.nice_node {{
    fill: rgba(0, 256, 0, 0.5);
    stroke: rgba(0, 256, 0, 0.9);
}}

.circle.bad_node {{
    stroke: rgba(256, 0, 0, 0.9);
    fill: rgba(256, 0, 0, 0.5);
}}

.link {{
    fill: none;
    stroke: #666;
    stroke-opacity: 0.2;
}}

#nice_target {{
    fill: green;
}}

.link.nice_target {{
    stroke: green;
}}

#source {{
    fill: yellow;
}}

.link.source {{
    stroke: #f3f310;
}}
                
.link.positive {{
    stroke: green;
}}
                
.link.negative {{
    stroke: red;
}}

#source {{
    fill: orange;
}}

.link.source1 {{
    stroke: orange;
}}

bad_target {{
    fill: red;
}}

.link.bad_target {{
    stroke: red;
}}

text {{
    font: 12px sans-serif;
    pointer-events: none;
    #text-shadow: 0 1px 0 #fff, 1px 0 0 #fff, 0 -1px 0 #fff, -1px 0 0 #fff;
}}
                
#mydiv {{
    position: absolute;
    text-align: center;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    background-color: transparent;
}}
                
.wrapper {{
    position: relative;
    width: 100%;
    height: 100%;
}}
                
.close {{
    position: absolute;
    right: 32px;
    top: 32px;
    width: 32px;
    height: 32px;
    opacity: 0.3;
}}
                
.close:hover {{
    opacity: 1;
}}
                
.close:before,
.close:after {{
    position: absolute;
    left: 15px;
    content: " ";
    height: 33px;
    width: 2px;
    background-color: #333;
}}
                
.close:before {{
    transform: rotate(45deg);
}}
                
.close:after {{
    transform: rotate(-45deg);
}}
                           
</style>

<script src="https://d3js.org/d3.v5.min.js"></script>
<script src="https://www.gstatic.com/charts/loader.js"></script>


<div>
  <input type="radio" class="radio" id="radioButton_mean" name="contact" value="mean"><label> Show mean </label>
  <input type="radio" class="radio" id="radioButton_min" name="contact" value="min"><label> Show min </label>
  <input type="radio" class="radio" id="radioButton_max" name="contact" value="max"><label> Show max </label>
</div>

<div class="parent" draggable={{true}}  id="mydiv"></div>
    <div class="wrapper">
        <div class="close" id="close-1"></div>

<table class="columns">
    </table>
<script>

var links = {links};

var node_params = {node_params};

var node_stats = {node_stats};

var nodes = {nodes};

var width = {width},
    height = {height};

var svg = d3.select("body").append("svg")
    .attr("width", width)
    .attr("height", height);
    
google.charts.load("current", {{packages:["corechart"]}});

console.log(svg.selectAll('.radio'));

let defs = svg.append("g").selectAll("marker")
    .data(links)
  .enter().append("marker")
    .attr("id", function(d) {{ return d.source.index + '-' + d.target.index; }})
    .attr("viewBox", "0 -5 10 10")
    .attr("refX", function(d) {{
        if (d.target.name !== d.source.name) {{
            return 7 + d.target.degree; 
        }} else {{
            return 0;
        }}
    }})
    .attr("refY", calcMarkers)
    .attr("markerWidth", 10)
    .attr("markerHeight", 10)
    .attr("markerUnits", "userSpaceOnUse")
    .attr("orient", "auto");

defs.append("path")
    .attr("d", "M0,-5L10,0L0,5");

function calcMarkers(d) {{
    let dist = Math.sqrt((nodes[d.target.index].x - nodes[d.source.index].x) ** 2 + (nodes[d.target.index].y - nodes[d.source.index].y) ** 2);
    if (dist > 0 && dist <= 200){{
        return - Math.sqrt((0.5 - (d.target.degree ) / 2 / dist)) * (d.target.degree) / 2;
    }} else {{
        return 0;
    }}
}}


var path = svg.append("g").selectAll("path")
    .data(links)
  .enter().append("path")
    .attr("class", function(d) {{ return "link " + d.type; }})
    .attr("stroke-width", function(d) {{ return Math.max(d.weight * 20, 1); }})
    .attr("marker-end", function(d) {{ return "url(#" + d.source.index + '-' + d.target.index + ")"; }})
    .attr("id", function(d,i) {{ return "link_"+i; }})
    .attr("parent_pk", function(d) {{ return d.source.pk; }})
    .attr("d", linkArc)
    ;

var edgetext = svg.append("g").selectAll("text")
    .data(links)
   .enter().append("text")
   .append("textPath")
    .attr("xlink:href",function(d,i){{return "#link_"+i;}})
    .style("text-anchor","right")
    .attr("startOffset", "40%")
    ;
    
function update() {{
    d3.selectAll(".radio").each(function(d) {{
        cb = d3.select(this);
        
        if (cb.property("checked")) {{
         radio_text = this.value;
         
            edgetext = edgetext.text(function(d) {{
                if (radio_text == "mean") {{
                    return d.weight_mean;
                }}
                
                 if (radio_text == "min") {{
                    return d.weight_min;
                }}
                
                 if (radio_text == "max") {{
                    return d.weight_max;
                }}
            }})
        }} 
        
        
    }})
}};

d3.selectAll(".radio").on("change", update);

function dragstarted(d) {{
  d3.select(this).raise().classed("active", true);
}}

function dragged(d) {{
  d3.select(this).attr("cx", d.x = d3.event.x).attr("cy", d.y = d3.event.y);
}}

function dragended(d) {{
  d3.select(this).classed("active", false);
  path = path.attr("d", linkArc);
  text = text
        .attr('x', function(d) {{ return d.x; }})
        .attr('y', function(d) {{ return d.y; }})
        ;
  defs = defs.attr("refY", calcMarkers);
  defs.append("path")
    .attr("d", "M0,-5L10,0L0,5");
}};

var circle = svg.append("g").selectAll("circle")
    .data(nodes)
  .enter().append("circle")
    .attr("class", function(d) {{ return "circle " + d.type; }})
    .attr("r", function(d) {{ return d.degree; }})
    .attr('cx', function(d) {{ return d.x; }})
    .attr('cy', function(d) {{ return d.y; }})
    .attr('id', function(d) {{ return d.pk; }})
    .call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));

var text = svg.append("g").selectAll("text")
    .data(nodes)
  .enter().append("text")
    .attr('x', function(d) {{ return d.x; }})
    .attr('y', function(d) {{ return d.y; }})
    .text(function(d) {{ return d.name; }});

function linkArc(d) {{
  var dx = nodes[d.target.index].x - nodes[d.source.index].x,
      dy = nodes[d.target.index].y - nodes[d.source.index].y,
      dr = dx * dx + dy * dy;
      dr = Math.sqrt(dr);
      if (dr > 200) {{
        dr *= 5
      }} else {{
        dr *= 5 /*dr /= 2*/
      }};
      if (dr > 0) {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + (dr * 1.1) + "," + (dr * 1.1) + " 0 0,1 " + nodes[d.target.index].x + "," + nodes[d.target.index].y;}}
      else {{return "M" + nodes[d.source.index].x + "," + nodes[d.source.index].y + "A" + 20 + "," + 20 + " 0 1,0 " + (nodes[d.target.index].x + 0.1) + "," + (nodes[d.target.index].y + 0.1);}}
}}

function downloadLayout() {{
    var a = document.createElement("a");
    var file = new Blob([JSON.stringify(nodes)], {{type: "text/json;charset=utf-8"}});
    a.href = URL.createObjectURL(file);
    a.download = "node_params.json";
    a.click();
}}

function draw_channels(ind) {{
       
    console.log([['Ветка', 'Процент']].concat(node_stats[ind]));
        
    var data = google.visualization.arrayToDataTable([['Ветка', 'Процент']].concat(node_stats[ind][0]));
 
 
    var options = {{pieResidueSliceLabel: 'Остальное',
                       width:500,
                       height:300,
                       backgroundColor:'transparent'
                       }};

    const parent = document.getElementById("mydiv");
    const new_parent = document.createElement("div");
        
    new_parent.setAttribute("draggable", "true")
    new_parent.setAttribute("id", "mydiv")
    document.body.appendChild(new_parent);
        
    dragElement(new_parent);
        
    const newChild = document.createElement("mydivheader_clone")
    newChild.setAttribute("mydivheader", "mydivheader_clone")
    new_parent.appendChild(newChild);
        
    var chart = new google.visualization.PieChart(newChild);
    chart.draw(data, options)
        
    const close = document.getElementById("close-1");

    close.onclick = e => {{
    const parent = document.getElementById("mydiv");
    parent.remove();
}};
        
}};
      
dragElement(document.getElementById("mydiv"));

function dragElement(elmnt) {{
  var pos1 = 0, pos2 = 0, pos3 = 0, pos4 = 0;
  if (document.getElementById(elmnt.id + "header")) {{
    document.getElementById(elmnt.id + "header").onmousedown = dragMouseDown;
  }} else {{
    elmnt.onmousedown = dragMouseDown;}}
    
function dragMouseDown(e) {{
  e = e || window.event;
  e.preventDefault();
  
  pos3 = e.clientX;
  pos4 = e.clientY;
  
  document.onmouseup = closeDragElement;
  document.onmousemove = elementDrag;
}}

function elementDrag(e) {{
  e = e || window.event;
  e.preventDefault();
  
  pos1 = pos3 - e.clientX;
  pos2 = pos4 - e.clientY;
  pos3 = e.clientX;
  pos4 = e.clientY;
  
  elmnt.style.top = (elmnt.offsetTop - pos2) + "px";
  elmnt.style.left = (elmnt.offsetLeft - pos1) + "px";
}}

function closeDragElement() {{
  document.onmouseup = null;
  document.onmousemove = null;
}}

}}


function set_visibility(next_style, id)
{{
    if (node_stats[id] != undefined) {{
 
svg.selectAll("circle").filter(obj => {{return node_stats[id][1].indexOf(obj.pk) != -1}}).style('visibility',next_style);
path.filter(obj => {{return String(id) == obj.parent_pk}}).style('visibility',next_style);

path.filter(obj => {{ return obj.source.pk == id}}).style('visibility', next_style);

text.filter(obj => {{return node_stats[id][1].indexOf(obj.pk) != -1}}).style('visibility', next_style);

edgetext.filter(obj => {{return obj.source.pk == id}}).style('visibility', next_style);

}}

}}
 
 
svg.selectAll("circle").on("dblclick", function () 
{{

console.log(node_stats[this.id]);

draw_channels(this.id);

curr_style = svg.selectAll("circle").filter(obj => {{return node_stats[this.id][1].indexOf(obj.pk) != -1}})._groups[0][0].style['visibility'];

if (curr_style == 'visible')
next_style = 'hidden';
else
next_style = 'visible';

let to_process = [this.id];
let all_children = [];

while (to_process.length > 0)
{{
    curr_id = to_process.pop();
    if (node_stats[curr_id] != undefined)
    {{
        for (var i = 0; i < node_stats[curr_id][1].length; i++)
        {{
            to_process.push(node_stats[curr_id][1][i]);
        }}
    }}    
    all_children.push(curr_id);
}}

 for (var i = 0; i < all_children.length; i++)
{{
    set_visibility(next_style, all_children[i]);
}}


}});



</script>
"""

In [87]:
node_params = {
                'positive_target_event': 'nice_target',
                'negative_target_event': 'bad_target',
                'source_event': 'source',
            }
node_stats = {}

In [88]:
nodes = [{'pk': 1, 'index': 0, 'name': '1', 'x': 600, 'y': 500, 'type': 'bad_node', 'degree': 4},
        {'pk': 2, 'index': 1, 'name': '2', 'x': 800, 'y': 500,  'type': 'nice_node', 'degree': 4},
        {'pk': 3, 'index': 2, 'name': '3', 'x': 400, 'y': 300,  'type': 'suit', 'degree': 4}]

In [89]:
links = [{'source': nodes[0], 'target': nodes[1], 'weight': 0.4, 'weight_mean': 'mean', 
          'weight_min': 'min', 'weight_max': 'max', 'type': 'nice_target'},
        {'source': nodes[2], 'target': nodes[0], 'weight': 0.4, 'weight_mean': 'mean', 
          'weight_min': 'min', 'weight_max': 'max', 'type': 'bad_target'},
        {'source': nodes[2], 'target': nodes[1], 'weight': 0.1, 'weight_mean': 'mean', 
          'weight_min': 'min', 'weight_max': 'max', 'type': 'suit'}]

In [90]:
x = __TEMPLATE__.format(
        width=1000,
        height=1000,
        links=json.dumps(links).encode('latin1').decode('utf-8'),
        node_params=json.dumps(node_params).encode('latin1').decode('utf-8'),
        nodes=json.dumps(nodes).encode('latin1').decode('utf-8'),
        node_stats=json.dumps(node_stats).encode('latin1').decode('utf-8')
        )

In [91]:
with open('test1.html', 'w') as f:
        f.write(x)

In [92]:
display(IFrame('test1.html', width=1000, height=1200))